In [ ]:
import pyautogui
import pyperclip
import time
import pandas as pd
from pathlib import Path
import warnings

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkcalendar import DateEntry

warnings.filterwarnings('ignore')

smss = tk.Tk()

smss.title('Enviar SMS')
smss.geometry("%dx%d+%d+%d" % (270,140,500,300))

label_escolher_data = tk.Label(anchor='n',text='O SMS é referente aos pacientes de qual dia? ', borderwidth=5, bg='#CCCCCC')
label_escolher_data.grid(row=0, column=0, columnspan=1, padx=10, pady=10, sticky='NSEW')

data_relatorio = DateEntry(year=2022, locale='pt_br')
data_relatorio.grid(row=1, column=0, columnspan=1, padx=10, pady=10, sticky='W')


def sms():
    pyautogui.PAUSE = 0.1
    messagebox.showwarning(title='ATENÇÃO',message="Favor colocar o asa MAXIMIZADO de fundo!")
    time.sleep(2)

    #ir até aba SMS
    pyautogui.click(x=42, y=73)
    pyautogui.click(x=20, y=154)
    pyautogui.click(x=64, y=249)
    pyautogui.click(x=164, y=251)
    time.sleep(1)

    #colocar data do dia seguinte XXX
    pyautogui.doubleClick(x=183, y=180)
    pyautogui.write(data_relatorio.get())
    pyautogui.press('tab')
    pyautogui.write(data_relatorio.get())

    #exportar relatório e salvar
    pyautogui.click(x=936, y=140)
    time.sleep(2)
    pyautogui.hotkey('alt','i')
    pyautogui.press('enter')
    time.sleep(4)
    pyautogui.click(x=14, y=34)
    time.sleep(2)
    pyautogui.click(x=461, y=464)
    pyautogui.click(x=691, y=487)
    pyautogui.click(x=588, y=587)
    pyautogui.click(x=519, y=521)
    pyperclip.copy('C:/Users/Administrator/Desktop/smsbruto.XLS')
    pyautogui.hotkey('ctrl','v')
    pyautogui.press('enter')
    time.sleep(4)
    pyautogui.hotkey('alt','F4')
    pyautogui.press('tab')
    pyautogui.press('tab')
    pyautogui.press('enter')
    pyautogui.hotkey('win','d')

    x = pd.read_excel(r'C:\Users\Administrator\Desktop\smsbruto.xls')
    
    # Pegar Linha [2] da planilha
    novo_header = x.iloc[2]

    # Transformar essa linha [2] em lista
    novo_header = list(novo_header)

    # colocar essa lista como cabeçalho
    x.columns = novo_header

    # escolher colunas uteis da planilha com base no NOME delas
    x = x[['Telefone 1','Telefone 2','Telefone 3']]
    x.columns = ['TELEFONE1','TELEFONE2','TELEFONE3']

    #Transformar colunas em lista
    y = list(x['TELEFONE1'])
    yb = list(x['TELEFONE2'])
    ybc = list(x['TELEFONE3'])

    #Retirar itens duplicados
    y = set(y)
    y = list(y)

    yb = set(yb)
    yb = list(yb)

    ybc = set(ybc)
    ybc = list(ybc)

    # Retirar 'Telefone 1'
    y.remove('Telefone 1')
    yb.remove('Telefone 2')
    ybc.remove('Telefone 3')

    # Agrupar as 3 colunas em uma só
    y.extend(yb)
    y.extend(ybc)

    #Transformar todos os elementos da lista em string
    for i, x in enumerate(y):
        y[i] = str(x)

    # Retirar )(-/
    nova = []

    for x in y:
        item = x
        for z in ['(', ')', '-', '/',' ']:
            item = item.replace(z, '')
        nova.append(item)

    # Retirar os NaN e as celulas que não tem 11 digitos  
    nova2 = []

    for x in nova:
        if len(x) == 11:
            nova2.append(x)

    # Retirar células que não começam com 21 9        
    nova3 = []

    for x in nova2:
        if x[0:3] == '219':
            nova3.append(x)

    # Colocar o 'TELEFONE' e 'NOME'      
    nome = []

    qtdnova3 = len(nova3)

    for x in range(qtdnova3):
        nome.append('NOME')

    dicionario = dict(TELEFONE = nova3, NOME = nome)

    planilha = pd.DataFrame(dicionario)

    #Excluir coluna de index
    planilha = planilha.set_index('TELEFONE')

    # Salvar planilha que será exportada
    planilha.to_excel(r'C:\Users\Administrator\Desktop\sms.xls')

    #login 00004475
    #senha cmad4475

    #logar no site da sonax
    servico = Service(ChromeDriverManager().install())
    navegador = webdriver.Chrome(service=servico)
    navegador.get('https://supervisor.sonax.net.br/login')
    time.sleep(5)
    navegador.maximize_window()
    navegador.find_element(By.XPATH,'/html/body/div[3]/form/div[2]/input').send_keys('#Login')
    navegador.find_element(By.XPATH,'/html/body/div[3]/form/div[3]/input').send_keys('#Senha')
    navegador.find_element(By.XPATH,'/html/body/div[3]/form/div[4]/button').click()
    time.sleep(5)
    navegador.get('https://supervisor.sonax.net.br/sms/relatorios')
    time.sleep(3)
    navegador.find_element(By.XPATH,'/html/body/div[1]/div[2]/form/div[1]/div[5]/input[2]').click()
    time.sleep(2)
    navegador.find_element(By.XPATH,'/html/body/div[1]/div[2]/form/div[3]/input[2]').click()
    time.sleep(3)
    
    #VERIFICAR
    pyperclip.copy('C:\\Users\\Administrator\\Desktop\\sms')
    pyautogui.hotkey('ctrl','v')
    time.sleep(1)
    pyautogui.press('enter')
    
    time.sleep(2)
    navegador.find_element(By.XPATH,'/html/body/div[3]/div[3]/div/button[1]/span').click()
    navegador.find_element(By.XPATH,'/html/body/div[4]/div[3]/div/button[1]/span').click()
    time.sleep(3)
    navegador.find_element(By.XPATH,'//*[@id="listaCampoExcel_0"]').click()
    navegador.find_element(By.XPATH,'//*[@id="listaCampo_NRO_DDD_TELEFONE"]').click()
    navegador.find_element(By.XPATH,'//*[@id="ContatoForm"]/div[3]/div[3]/input[1]').click()
    navegador.find_element(By.XPATH,'//*[@id="listaCampoExcel_1"]').click()
    navegador.find_element(By.XPATH,'//*[@id="listaCampo_NOM_NOME"]').click()
    navegador.find_element(By.XPATH,'//*[@id="ContatoForm"]/div[3]/div[3]/input[1]').click()
    navegador.find_element(By.XPATH,'//*[@id="ContatoForm"]/div[5]/input[2]').click()
    time.sleep(3)
    navegador.find_element(By.XPATH,'//*[@id="campanhaVo.dscCampanha"]').send_keys('sms')
    time.sleep(2)
    navegador.find_element(By.XPATH,'//*[@id="campanhaVo.txtMensagem"]').send_keys('Confirmamos seu atendimento conosco para amanhã. Clínica Gerson Schorr de Alergia e Dermatologia (21) 2220-5798. Link para confirmação: ')
    time.sleep(2)
    
    time.sleep(2)
    #seta pra baixo e consulta
    navegador.find_element(By.XPATH,'//*[@id="campanhaVo.indWhatsapp1"]').click()
    time.sleep(2)
    navegador.find_element(By.XPATH,'//*[@id="campanhaVo.telefoneWhatsapp"]').send_keys('21984360981')
    time.sleep(2)
    navegador.find_element(By.XPATH,'//*[@id="campanhaVo.txtWhatsapp"]').send_keys('Consulta confirmada!')
    navegador.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(3)
    navegador.find_element(By.XPATH,'//*[@id="btn_gravar"]').click()
    time.sleep(10)
    navegador.refresh()
    sms = navegador.find_element(By.ID,'info').text
    sms = sms.split('4475')[1]
    sms = sms.split('Alterar')[0]
    sms = sms.split('Sms')[0]
    sms = int(sms.split('|')[1])
    if sms < 500:
        messagebox.showwarning(title='ATENÇÃO',message='Fazer pedido de créditos SMS')
    messagebox.showinfo(title='ATENÇÃO',message='Agora é só enviar a campanha!')
    
    apagarsms = Path(r'C:\Users\Administrator\Desktop\sms.xls')
    apagarsmsbruto = Path(r'C:\Users\Administrator\Desktop\smsbruto.xls')

    apagarsms.unlink()
    apagarsmsbruto.unlink()

botao_enviar_sms = tk.Button(text='OK', command=sms, height=1, width=20)
botao_enviar_sms.grid(row=9, column=0, columnspan=1, padx=10, pady=10)

smss.mainloop()




====== WebDriver manager ======
Current google-chrome version is 108.0.5359
Get LATEST chromedriver version for 108.0.5359 google-chrome
Driver [C:\Users\Administrator\.wdm\drivers\chromedriver\win32\108.0.5359.71\chromedriver.exe] found in cache
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Administrator\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_3544/3811664352.py", line 184, in sms
    navegador.find_element(By.XPATH,'/html/body/div[3]/div[3]/div/button[1]/span').click()
  File "C:\Users\Administrator\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 1238, in find_element
    return self.execute(Command.FIND_ELEMENT, {
  File "C:\Users\Administrator\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 418, in execute
    self.error_handler.check_response(response)
  File "C:\Users\Administrator\anaconda3

In [ ]:
x = navegador.find_element(By.XPATH,'//*[@id="comboCentroCusto"]').text

print(x)

#### Lógica Para Filtro de Data

In [ ]:
if '/' in date:
    x = date.replace('/','')C:C:/Users/Administrator/Desktop/smsbruto.XLS
            /C:/Users/Administrator/Desktop/smsbruto.XLS
            C:/Users/Administrator/Desktop/smsbruto.XLS
            Users/Administrator/Desktop/smsbruto.XLS
        
    if x.isnumeric() and len(x) == 8:
        sms()
        print(date)
    else:
        pyautogui.alert('Data incorreta, Por Favor Digite Novamente!')
elif date.isnumeric() and len(date) == 8:
    sms()
    print(date)
else:
    pyautogui.alert('Data incorreta, Por Favor Digite Novamente!')    

### Para dias em que sejam necessários filtros na planilha

In [3]:
import pyautogui
import pyperclip
import time
import pandas as pd
from pathlib import Path
import warnings

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

x = pd.read_excel(r'C:\Users\Administrator\Desktop\smsbrutoa.xlsx')

#Excluir Colunas inúteis
x = x[['Unnamed: 9','Unnamed: 10','Unnamed: 11']]

#Renomear colunas
x.columns = ['TELEFONE1','TELEFONE2','TELEFONE3']

#Transformar colunas em lista
y = list(x['TELEFONE1'])
yb = list(x['TELEFONE2'])
ybc = list(x['TELEFONE3'])

#Retirar itens duplicados
y = set(y)
y = list(y)

yb = set(yb)
yb = list(yb)

ybc = set(ybc)
ybc = list(ybc)

# Retirar 'Telefone 1'
y.remove('Telefone 1')
yb.remove('Telefone 2')
ybc.remove('Telefone 3')

# Agrupar as 3 colunas em uma só
y.extend(yb)
y.extend(ybc)

#Transformar todos os elementos da lista em string
for i, x in enumerate(y):
    y[i] = str(x)

# Retirar )(-/
nova = []

for x in y:
    item = x
    for z in ['(', ')', '-', '/',' ']:
        item = item.replace(z, '')
    nova.append(item)

# Retirar os NaN e as celulas que não tem 11 digitos  
nova2 = []

for x in nova:
    if len(x) == 11:
        nova2.append(x)

# Retirar células que não começam com 21 9        
nova3 = []

for x in nova2:
    if x[0:3] == '219':
        nova3.append(x)

# Colocar o 'TELEFONE' e 'NOME'      
nome = []

qtdnova3 = len(nova3)

for x in range(qtdnova3):
    nome.append('NOME')

dicionario = dict(TELEFONE = nova3, NOME = nome)

planilha = pd.DataFrame(dicionario)

#Excluir coluna de index
planilha = planilha.set_index('TELEFONE')

# Salvar planilha que será exportada
planilha.to_excel(r'C:\Users\Administrator\Desktop\sms.xls')

#login 00004475
#senha cmad4475

#logar no site da sonax
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)
navegador.get('https://supervisor.sonax.net.br/login')
time.sleep(5)
navegador.maximize_window()
navegador.find_element(By.XPATH,'/html/body/div[3]/form/div[2]/input').send_keys('#Login')
navegador.find_element(By.XPATH,'/html/body/div[3]/form/div[3]/input').send_keys('#Senha')
navegador.find_element(By.XPATH,'/html/body/div[3]/form/div[4]/button').click()
time.sleep(5)
navegador.get('https://supervisor.sonax.net.br/sms/relatorios')
time.sleep(3)
navegador.find_element(By.XPATH,'/html/body/div[1]/div[2]/form/div[1]/div[5]/input[2]').click()
time.sleep(2)
navegador.find_element(By.XPATH,'/html/body/div[1]/div[2]/form/div[3]/input[2]').click()
time.sleep(3)

#VERIFICAR
pyperclip.copy('C:\\Users\\Administrator\\Desktop\\sms')
pyautogui.hotkey('ctrl','v')
time.sleep(1)
pyautogui.press('enter')

time.sleep(2)
navegador.find_element(By.XPATH,'/html/body/div[3]/div[3]/div/button[1]/span').click()
navegador.find_element(By.XPATH,'/html/body/div[4]/div[3]/div/button[1]/span').click()
time.sleep(3)
navegador.find_element(By.XPATH,'//*[@id="listaCampoExcel_0"]').click()
navegador.find_element(By.XPATH,'//*[@id="listaCampo_NRO_DDD_TELEFONE"]').click()
navegador.find_element(By.XPATH,'//*[@id="ContatoForm"]/div[3]/div[3]/input[1]').click()
navegador.find_element(By.XPATH,'//*[@id="listaCampoExcel_1"]').click()
navegador.find_element(By.XPATH,'//*[@id="listaCampo_NOM_NOME"]').click()
navegador.find_element(By.XPATH,'//*[@id="ContatoForm"]/div[3]/div[3]/input[1]').click()
navegador.find_element(By.XPATH,'//*[@id="ContatoForm"]/div[5]/input[2]').click()
time.sleep(3)
navegador.find_element(By.XPATH,'//*[@id="campanhaVo.dscCampanha"]').send_keys('sms')
time.sleep(2)
navegador.find_element(By.XPATH,'//*[@id="campanhaVo.txtMensagem"]').send_keys('Confirmamos seu atendimento conosco para amanhã. Clínica Gerson Schorr de Alergia e Dermatologia (21) 2220-5798. Link para confirmação: ')
time.sleep(2)

time.sleep(2)
#seta pra baixo e consulta
navegador.find_element(By.XPATH,'//*[@id="campanhaVo.indWhatsapp1"]').click()
time.sleep(2)
navegador.find_element(By.XPATH,'//*[@id="campanhaVo.telefoneWhatsapp"]').send_keys('21984360981')
time.sleep(2)
navegador.find_element(By.XPATH,'//*[@id="campanhaVo.txtWhatsapp"]').send_keys('Consulta confirmada!')
navegador.execute_script("window.scrollTo(0,document.body.scrollHeight)")
navegador.find_element(By.XPATH,'//*[@id="btn_gravar"]').click()
time.sleep(5)
sms = navegador.find_element(By.ID,'info').text
sms = sms.split('4475')[1]
sms = sms.split('Alterar')[0]
sms = sms.split('Sms')[0]
sms = int(sms.split('|')[1])
if sms < 500:
    messagebox.showwarning(title='ATENÇÃO',message='Fazer pedido de créditos SMS')
messagebox.showinfo(title='ATENÇÃO',message='Agora é só enviar a campanha!')

apagarsms = Path(r'C:\Users\Administrator\Desktop\sms.xls')
apagarsmsbruto = Path(r'C:\Users\Administrator\Desktop\smsbruto.xls')

apagarsms.unlink()
apagarsmsbruto.unlink()




====== WebDriver manager ======
Current google-chrome version is 108.0.5359
Get LATEST chromedriver version for 108.0.5359 google-chrome
Driver [C:\Users\Administrator\.wdm\drivers\chromedriver\win32\108.0.5359.71\chromedriver.exe] found in cache


FileNotFoundError: [WinError 2] O sistema não pode encontrar o arquivo especificado: 'C:\\Users\\Administrator\\Desktop\\smsbruto.xls'